# Vizier Auto-tuning

Use this notebook to find optimal parameters for `nextpnr` using Google Vizier framework.

###  Install python dependencies

In [ ]:
! pip install -U google-api-python-client
! pip install -U google-cloud
! pip install -U google-cloud-storage
! pip install -U requests
! pip install -U matplotlib
! pip install -U scipy

### Synthesize design

Let's synthesize design beforehand. After all we want to test only nextpnr.

In [ ]:
import os

# Specify a path for your riscv toolchain installation.
# Might be unncessary if you are running inside a conda environment
riscv_toolchain_path = '../../../riscv64-unknown-elf-gcc-10.1.0-2020.08.2-x86_64-linux-ubuntu14/'
riscv_toolchain_path = os.path.realpath(riscv_toolchain_path)

rv_bin = os.path.join(riscv_toolchain_path, 'bin')
path = f'{rv_bin}:{os.environ["PATH"]}'

! source ../../env/conda/bin/activate cfu-common
%env PATH $path
! make synth

### Configure SaaS backend and authorization

In [ ]:
# Google Cloud Vizier config
gc_accnt = 'example@email.com'
gc_project_id = 'project_id'
gc_region= 'region'
%env GOOGLE_APPLICATION_CREDENTIALS credentials.json

### Create study

In [ ]:
import random
import datetime
from vizier.vapi import *

random.seed()

def make_study_name(identity):
    random_id = ''.join(str(random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])) for i in range(0, 8))
    return f'hps-study{random_id}_{identity}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'

study_id = make_study_name('hps')

print(f'Study will be named "{study_id}"')

study = Study(
    name = study_id,
    max_trials = 3,
    # Add metrics to optimize here
    metrics = {
        'fmax': 'MAXIMIZE',
        'runtime': 'MINIMIZE'
    },
    # Add parameters for the blackbox here.
    # Use 'UNSPECIFIED'/'LENEAR'/'LOG'/'REVERSE_LOG' to define parameter scaling (Vizier)
    parameters = {
        'estimate-delay-mult': ParameterInt(min=10, max=50, scale='LINEAR')
    },
    # Choose algorithm. Vizier accepts the following options:
    # 'ALGORITHM_UNSPECIFIED', 'GAUSSIAN_PROCESS_BANDIT', 'GRID_SEARCH' (discrete/categorical params only)
    # and 'RANDOM_SEARCH'
    algorithm = 'GAUSSIAN_PROCESS_BANDIT'
)

### Create a Vizier-compatible client

This version of script is meant to be used with Vizier backend. Use `VizierClient`.

In [ ]:
from vizier.client_vizier import VizierClient

! gcloud config set project $gc_project_id
# ! gcloud auth application-default login
! gcloud auth login $gc_accnt
client = VizierClient(gc_accnt, gc_project_id, gc_region)

### Set-up an Optimizer for HPS flow

This is where we define how to use out "black-box", ie. the Oxide flow

In [ ]:
from vizier import Optimizer
import subprocess
import os
import shutil
import json
import time

# Path to nextpnr-nexus
nextpnr = '/mnt/more/Documents/symbiflow-enhancements-fpga-tool-perf/env/conda'\
          '/envs/nextpnr-env/bin/nextpnr-nexus'

def get_metrics_from_report(json_dict):
    return {
        'fmax': json_dict['fmax']['clkout$glb_clk']['achieved']
    }

def params_to_npr_args(params):
    args = []
    for name, value in params.items():
        args += [f'--{name}', str(value)]
    return args

def standard_npr_args(json, pdc, result_fasm, device):
    return [
        '--json', json,
        '--pdc', pdc,
        '--fasm', result_fasm,
        '--device', device,
        '--detailed-timing-report'
    ]

next_dir = 'next'
os.makedirs(next_dir, exist_ok=True)

gateware_dir = os.path.realpath('../../soc/build/hps.hps_accel/gateware')

class HpsFlowOptimizer(Optimizer):

    # Run the blackbox inside the `run_blackbox` method. The `instance` parameter is an instance
    # number of the blackbox if multiple blackboxes are being run in parallel. This can be used
    # avoid problems if the blackbox generates some artifacts in the filesystem.
    def run_blackbox(self, instance, **params) -> 'list[Measurement]':
        dir_path = os.path.join(next_dir, f'run_{instance}')

        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
        os.makedirs(dir_path)

        report_path = os.path.join(dir_path, f'next_report.json')

        args = standard_npr_args(
            json = os.path.join(gateware_dir, 'hps_proto2_platform.json'),
            pdc = os.path.join(gateware_dir, 'hps_proto2_platform.pdc'),
            result_fasm = os.path.join(dir_path, 'hps_proto2_platform.fasm'),
            device = 'LIFCL-17-8UWG72C'
        ) + ['--report', report_path] + params_to_npr_args(params)

        now = time.time()
        subprocess.run([nextpnr] + args)
        duration = time.time() - now

        metrics: dict
        with open(report_path, 'r') as report:
            metrics = get_metrics_from_report(json.loads(report.read()))
        metrics['runtime'] = duration
        
        return [Measurement(metrics = metrics)]

### Auto-tune

In [ ]:
from vizier import Optimizer

concurrency = 1

client.create_study(study, concurrency)

# Best trial is currently being calculated by finding the longest vector.
# In case of values that are expected to be minimized, they get substracted from maximum during
# computations. See `get_best_trial` in cielnt_vizier.py.
best_trial = HpsFlowOptimizer(study, concurrency).optimize(client)

print(f'Parameters: {", ".join(["{}: {}".format(p.name, p.value) for p in best_trial.parameters])}')

i = 0
for measurement in best_trial.measurements:
    print(f'Measurements[{i}]: {",".join(["{}: {}".format(name, value) for name, value in measurement.metrics.items()])}')
    i += 0